# LANE CHANGE
María Rodríguez Palomo

In [1]:
import glob
import os
import sys
import carla
import argparse
import random
import time
import math
import numpy as np
import cv2
import keyboard as kb


#### CREATE WORLD

In [13]:
client = carla.Client('localhost', 2000)
client.set_timeout(2000)
#client.load_world("Town05")
world = client.get_world()
map = world.get_map()

#### CHANGE VIEW

In [14]:
loc2=carla.Transform(carla.Location(x=-200.229767, y=197.991257, z=11.914517), carla.Rotation(pitch=0.0, yaw=-100.194565, roll=0.006472))
spectator = world.get_spectator() 
spectator_pos = carla.Transform(loc2.location + carla.Location(x=-5,y=-40,z=70),carla.Rotation(yaw=loc2.rotation.yaw+10,pitch=loc2.rotation.pitch-80))
spectator.set_transform(spectator_pos)

#### CREATE VEHICLE

In [15]:
loc2=carla.Transform(carla.Location(x=-174.557922, y=188.525208, z=13.138501), carla.Rotation(pitch=0.733207, yaw=-147.194565, roll=0.006472))
blueprint_library = world.get_blueprint_library()
bp = random.choice(blueprint_library.filter('vehicle.tesla.model3'))

if bp.has_attribute('color'):
    color = bp.get_attribute('color').recommended_values[2]
    bp.set_attribute('color', '255,0,255')

vehicle = world.spawn_actor(bp, loc2)

#### CONSTANT SPEED

In [16]:
def maintain_speed(s,preferred_speed,speed_threshold):
    if s>=preferred_speed:
        return 0
    elif s < preferred_speed - speed_threshold:
        return 0.8
    else:
        return 0.4

#### ANGLE

In [17]:
def angle_between(v1, v2):
    return math.degrees(np.arctan2(v1[1], v1[0]) - np.arctan2(v2[1], v2[0]))

def get_angle(car,wp):
    vehicle_pos = car.get_transform()
    car_x = vehicle_pos.location.x
    car_y = vehicle_pos.location.y
    wp_x = wp.transform.location.x
    wp_y = wp.transform.location.y
    
    x = (wp_x - car_x)/((wp_y - car_y)**2 + (wp_x - car_x)**2)**0.5
    y = (wp_y - car_y)/((wp_y - car_y)**2 + (wp_x - car_x)**2)**0.5
    
    car_vector = vehicle_pos.get_forward_vector()
    degrees = angle_between((x,y),(car_vector.x,car_vector.y))
    return degrees

In [19]:
def steer(predicted_angle,MAX_STEER_DEGREES):
    if predicted_angle<-180:
        predicted_angle = predicted_angle+360
    elif predicted_angle > 180:
        predicted_angle = predicted_angle-360
    steer_input = predicted_angle
    if predicted_angle<-MAX_STEER_DEGREES:
        steer_input = -MAX_STEER_DEGREES
    elif predicted_angle>MAX_STEER_DEGREES:
        steer_input = MAX_STEER_DEGREES
    steer_input = steer_input/75
    return steer_input

#### DRIVING

Restart the kernel each time for better performance

In [ ]:
finalizar_der=1 
finalizar_izq=1 
a=True
b=True
preferred_speed = 30
speed_threshold = 2
MAX_STEER_DEGREES = 30

while True:
    loc=vehicle.get_location()
    current_w = map.get_waypoint(loc)
    
    #Lane change
    if kb.is_pressed("r") or finalizar_der==0: #If we press the key and haven't changed lanes yet
        finalizar_der=0
        if kb.is_pressed("r") and a:#If we press the key, we only want it to trigger once
            lane = current_w.lane_id+1
            a=False
        if lane == 5: #If we can't go further to the right anymore
            print("No se puede ir más a la derecha")
            finalizar_der=1
            a=True
        if current_w.lane_id !=lane: #If we are not yet in the lane we want
            next_w0 = map.get_waypoint_xodr(road_id=next_w.road_id,lane_id=next_w.lane_id+1,s=next_w.s)
            next_w = map.get_waypoint(next_w0.transform.location,project_to_road=True, lane_type=(carla.LaneType.Driving))
        else:#If we are already in the lane
            print("Hemos llegado al carril")
            finalizar_der=1
            a=True
            
    elif kb.is_pressed("l") or finalizar_izq==0: 
        finalizar_izq=0
        if kb.is_pressed("l") and b:
            lane = current_w.lane_id-1
            b=False
        if lane == 1: 
            print("No se puede ir más a la izquierda")
            finalizar_izq=1
            b=True
        if current_w.lane_id !=lane: 
            next_w0 = map.get_waypoint_xodr(road_id=next_w.road_id,lane_id=next_w.lane_id-1,s=next_w.s)
            next_w = map.get_waypoint(next_w0.transform.location,project_to_road=True, lane_type=(carla.LaneType.Driving))
        else:
            print("Hemos llegado al carril")
            finalizar_izq=1
            b=True
            
    else:
        waypoint_separation = 3
        next_w0 = list(current_w.next(waypoint_separation))[0]
        next_w = map.get_waypoint(next_w0.transform.location,project_to_road=True, lane_type=(carla.LaneType.Driving))
    
    velocity = vehicle.get_velocity()
    speed = round(3.6 *math.sqrt(velocity.x**2+ velocity.y**2+velocity.z**2),0)
    estimated_throttle = maintain_speed(speed,preferred_speed,speed_threshold)
    predicted_angle = get_angle(vehicle,next_w)
    steer_input=steer(predicted_angle,MAX_STEER_DEGREES)
    if steer_input > 0.05:
        estimated_throttle=0
    vehicle.apply_control(carla.VehicleControl(throttle= estimated_throttle, steer=steer_input))
    

#### DESTROY

In [22]:
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()